In [1]:
import numpy as np
import scipy as sp
import time, sys
import matplotlib.pyplot as plt
from scipy.sparse.linalg import spsolve

mu1,mu2=np.meshgrid(np.linspace(-1,-0.01,12),np.linspace(-1,-0.01,12),indexing='ij')
mu1=mu1.flatten()
mu2=mu2.flatten()
no_para=np.size(mu1,axis=0)

N=70
Ns=(N-1)**2
Nt=50
h=1/N
k=2/Nt
x1,x2=np.meshgrid(np.linspace(0,1,N+1)[1:-1],np.linspace(0,1,N+1)[1:-1],indexing='ij') # only free degrees of freedom
x1=x1.flatten()
x2=x2.flatten()
t=np.linspace(0,2,Nt+1)

e=np.ones(N-1)
A1D=1/h**2*sp.sparse.spdiags(np.vstack((e,-2*e,e)),[-1,0,1],N-1,N-1,format='csc')
A2D=sp.sparse.kron(A1D,sp.sparse.eye(N-1,format='csc'),format='csc')+sp.sparse.kron(sp.sparse.eye(N-1,format='csc'),A1D,format='csc')
I=sp.sparse.eye(Ns,format='csc')

# High-fidelity
U=np.zeros((Ns,no_para*Nt))
FOM_t_start=time.time()
for p in range(no_para):
    A=A2D-sp.sparse.diags(1/np.sqrt((x1-mu1[p])**2+(x2-mu2[p])**2),format='csc')
    f=(1/np.sqrt((x1-mu1[p])**2+(x2-mu2[p])**2).reshape(-1,1))@(np.sin(2*np.pi*t).reshape(1,-1))
    u=np.zeros((Ns,Nt+1))
    for n in range(Nt):
        u[:,n+1]=spsolve(I-k*A,u[:,n]+k*f[:,n+1])      
    U[:,np.arange(p*Nt,(p+1)*Nt)]=u[:,1:]
FOM_t_elapsed=time.time()-FOM_t_start
    
# POD of solution snapshot
W,S,VT=np.linalg.svd(U)

# Generate ROM basis
ns=20
PHIs=W[:,:ns]
PHIsT=PHIs.T

# construct Djk
nt=7
D=np.zeros((ns,nt*Nt))
for i in range(ns):
    Ri=VT[i,:]
    Ri=Ri.reshape(-1,no_para,order='F')
    Wi,Si,ViT=np.linalg.svd(Ri)
    PHIti=Wi[:,:nt]
    for j in range(nt):
        D[i,Nt*j:Nt*(j+1)]=PHIti[:,j]
        
# construct PHIst
PHIst=np.zeros((Ns*Nt,ns*nt))
for i in range(Nt):
    for j in range(nt):
        PHIstij=np.zeros((Ns,ns))
        Dij=sp.sparse.diags(D[:,j*Nt+i],format='csc')
        PHIst[Ns*i:Ns*(i+1),ns*j:ns*(j+1)]=PHIs@Dij  
PHIstT=PHIst.T

# Run Space-Time ROM for arbitrary parameter
mu=[-0.5,-0.5] # set parameter mu
A=A2D-sp.sparse.diags(1/np.sqrt((x1-mu[0])**2+(x2-mu[1])**2),format='csc')
f=(1/np.sqrt((x1-mu[0])**2+(x2-mu[1])**2).reshape(-1,1))@(np.sin(2*np.pi*t).reshape(1,-1))
# construct Ast and fst
As=PHIsT@A@PHIs
fs=PHIsT@f
Ast=np.zeros((ns*nt,ns*nt))
for i in range(nt):
    for j in range(nt):
        Astij=np.zeros((ns,ns))
        for kk in range(Nt):
            Dik=sp.sparse.diags(D[:,i*Nt+kk],format='csc')
            Djk=sp.sparse.diags(D[:,j*Nt+kk],format='csc')
            Astij+=Dik@Djk-k*Dik@As@Djk
        for kk in range(Nt-1):
            Dik_next=sp.sparse.diags(D[:,i*Nt+kk+1],format='csc')
            Djk=sp.sparse.diags(D[:,j*Nt+kk],format='csc')
            Astij-=Dik_next@Djk   
        Ast[ns*i:ns*(i+1),ns*j:ns*(j+1)]=Astij  
fst=np.zeros(ns*nt)
for j in range(nt):
    for kk in range(Nt):
        Djk=sp.sparse.diags(D[:,j*Nt+kk],format='csc')
        fst[ns*j:ns*(j+1)]+=k*Djk@fs[:,kk+1]
        
# Space-Time ROM
ROM_t_start=time.time()
UstROM=PHIst@np.linalg.solve(Ast,fst)
ROM_t_elapsed=time.time()-ROM_t_start

# Run HF model to check error when a parameter set is arbitary
mu=[-0.5,-0.5] # set parameter mu
A=A2D-sp.sparse.diags(1/np.sqrt((x1-mu[0])**2+(x2-mu[1])**2),format='csc')
f=(1/np.sqrt((x1-mu[0])**2+(x2-mu[1])**2).reshape(-1,1))@(np.sin(2*np.pi*t).reshape(1,-1))
HF_t_start=time.time()
u=np.zeros((Ns,Nt+1))
for n in range(Nt):
    u[:,n+1]=spsolve(I-k*A,u[:,n]+k*f[:,n+1])
UstHF=u[:,1:].flatten(order='F')
HF_t_elapsed=time.time()-HF_t_start

In [2]:
# Print results
print('max difference',np.max(UstHF-UstROM),'min difference',np.min(UstHF-UstROM))
print('relative error(%)',np.linalg.norm(UstROM-UstHF)/np.linalg.norm(UstHF)*100)
print('speed-up',HF_t_elapsed/ROM_t_elapsed)
print('High Fidelity Model', HF_t_elapsed)
print('ROM Model', ROM_t_elapsed)
print('training time', FOM_t_elapsed)

#compute residual
ee = np.ones(Nt-1)
lower_diag = sp.sparse.spdiags(np.vstack((1*ee,0*ee,0*ee)),[-1,0,1],Nt,Nt,format='csc') 
Abig_st=sp.sparse.kron(sp.sparse.eye(Nt,format='csc'),(I-k*A),format='csc') + \
       sp.sparse.kron(lower_diag,-I,format='csc') 
residual = Abig_st@UstROM-f[:,1:].flatten(order='F')
normalized_residual = residual/np.linalg.norm(residual)
print('Normalized Residual',np.max(np.abs(normalized_residual)))

max difference 4.447610722308967e-07 min difference -4.129098965946834e-07
relative error(%) 0.00023110232853934337
speed-up 53.85531866160218
High Fidelity Model 0.672706127166748
ROM Model 0.012490987777709961
training time 94.26536178588867
Normalized Residual 0.005360247884719481
